In [27]:
import os
import torch
import pandas as pd
import scanpy as sc
import numpy as np
from sklearn import metrics
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

from scipy.stats import pearsonr
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error as mse
from scipy.sparse import csr_matrix

from libpysal.weights import KNN
from esda.moran import Moran

from GraphST import GraphST

In [28]:
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")

In [29]:
dataset = 'Human_Lymph_Node'

In [30]:
file_fold = '/home/lytq/Spatial-Transcriptomics-Benchmark/data/' + dataset
adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True)
adata.var_names_make_unique()

In [31]:
adata_before = adata

In [32]:
model = GraphST.GraphST(adata, device=device)
adata = model.train()

AnnData object with n_obs × n_vars = 4035 × 36601
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'spatial', 'hvg', 'log1p'
    obsm: 'spatial'
    layers: 'count'
Begin to train ST data...
Debugging


100%|██████████| 600/600 [00:18<00:00, 32.69it/s]

Optimization finished for ST data!


In [33]:
adata_before

AnnData object with n_obs × n_vars = 4035 × 36601
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [34]:
adata_before.var_names

Index(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3',
       'AL627309.2', 'AL627309.5', 'AL627309.4', 'AP006222.2', 'AL732372.1',
       ...
       'AC133551.1', 'AC136612.1', 'AC136616.1', 'AC136616.3', 'AC136616.2',
       'AC141272.1', 'AC023491.2', 'AC007325.1', 'AC007325.4', 'AC007325.2'],
      dtype='object', length=36601)

In [35]:
adata_after = adata.obsm['emb']
orig_genes = adata_before.var_names.copy()
highly_variable_genes = adata.var_names[adata.var['highly_variable']]
adata_before = adata_before[:, highly_variable_genes].copy()


gene_df2 = pd.DataFrame(adata_after,
                        index=adata.obs.index,
                        columns=highly_variable_genes)

gene_df1 = pd.DataFrame(adata_before.X.toarray(),
                        index=adata.obs.index,
                        columns=adata_before.var.index)

In [36]:
gene_df2.shape, gene_df1.shape

((4035, 3000), (4035, 3000))

In [37]:
assert gene_df1.shape == gene_df2.shape

In [38]:
vec1 = gene_df1.values.flatten()
vec2 = gene_df2.values.flatten()

pcc, _ = pearsonr(vec1, vec2)
print(f"Pearson Correlation Coefficient: {pcc:.4f}")

Pearson Correlation Coefficient: 0.3186


In [39]:
arr1 = gene_df1.values
arr2 = gene_df2.values

ssim_value = ssim(arr1, arr2, data_range=arr2.max() - arr2.min())
print(f"Structural Similarity Index: {ssim_value:.4f}")

Structural Similarity Index: 0.5102


In [40]:
rmse_value = np.sqrt(mse(arr1, arr2))
print(f"Root Mean Squared Error: {rmse_value:.4f}")

Root Mean Squared Error: 22.1792


In [41]:
def compute_morans_i(exp_matrix, w):
    morans_i_list = np.empty(exp_matrix.shape[1])
    for i in range(exp_matrix.shape[1]): # Iterate over genes
        if i % 500 == 0: # Print progress every 100 genes
            print(i)
        morans_i_list[i] = Moran(exp_matrix[:, i], w).I
    return np.mean(morans_i_list) # Return the average Moran's I value


spatial_coords = adata.obsm['spatial']
w = KNN.from_array(spatial_coords, k=5)


In [42]:
morans_i_before = compute_morans_i(gene_df1.values, w)
morans_i_after = compute_morans_i(gene_df2.values, w)
print(f"Moran's I before: {morans_i_before:.4f}")
print(f"Moran's I after: {morans_i_after:.4f}")

0
500
1000
1500
2000
2500
0
500
1000
1500
2000
2500
Moran's I before: 0.0992
Moran's I after: 0.5713


In [43]:
results = {
    'Model': 'GraphST',
    'Dataset': dataset,
    "Moran's I before": morans_i_before,
    "Moran's I after": morans_i_after,
    'PCC': pcc,
    'SSIM': ssim_value,
    'RMSE': rmse_value
}

output_path = '/home/lytq/Spatial-Transcriptomics-Benchmark/Results/Imputation/GraphST/metrics.csv'
df_results = pd.DataFrame([results])
df_results.to_csv(output_path, mode='a', header=False, index=False)